In [ ]:
import numpy as np
from uncertainties import unumpy as unp
import matplotlib.pyplot as plt
import copy
import pisa
from pisa.core.detectors import Detectors
from pisa.core.distribution_maker import DistributionMaker
from pisa.core.pipeline import Pipeline
from pisa.analysis.analysis import Analysis
from pisa import FTYPE, ureg

We need to define two different detectors. To make things easy (and use existing cfg files) we set up two 3y DeepCore's but call them `detector1` and `detector2`. In general the Detectors class (just like the DistributionMaker class) accepts the cfg path strings as input, but since we want to modify the Pipelines here we have to initialize them first.

In [ ]:
p1_nu = Pipeline("settings/pipeline/IceCube_3y_neutrinos.cfg")
p1_mu = Pipeline("settings/pipeline/IceCube_3y_muons.cfg")
p1_nu.detector_name, p1_mu.detector_name = 'detector1', 'detector1'

p2_nu = Pipeline("settings/pipeline/IceCube_3y_neutrinos.cfg")
p2_mu = Pipeline("settings/pipeline/IceCube_3y_muons.cfg")
p2_nu.detector_name, p2_mu.detector_name = 'detector2', 'detector2'

Now we can initialize the Detectors class. We define all free parameters except the effective area as shared parameters. So there will be two effective areas (one for each detector).

In [ ]:
model = Detectors([p1_nu, p1_mu, p2_nu, p2_mu], shared_params=['deltam31', 'theta13', 'theta23', 'nue_numu_ratio', 'Barr_uphor_ratio', 'Barr_nu_nubar_ratio', 'delta_index', 'nutau_norm', 
                                                               'nu_nc_norm', 'opt_eff_overall', 'opt_eff_lateral', 'opt_eff_headon', 'ice_scattering', 'ice_absorption', 'atm_muon_scale'])
# this just turns on profiling
model.profile = True
model

Our model has a number of free parameters, that will be used in our fit to the data. aeff_scale appears two times because it is not a shared parameter. If a parameter has the same name for two detectors but is not shared, the detector name is added to the parameter for all but the first detector.

In [ ]:
model.params.free

The two distribution makers are completely similar but have a different name (as intended)

In [ ]:
model.distribution_makers[0]

In [ ]:
model.distribution_makers[1]

The two pipelines are quite different, with most complexity in the neutrino pipeline, that has several `Stage`s and free parameters:

In [ ]:
model.distribution_makers[0].pipelines[0]

In [ ]:
model.distribution_makers[0].pipelines[0].stages[2].params

While the muon pipleine is rather simple

In [ ]:
model.distribution_makers[0].pipelines[1]

## Retrieve Outputs

We can get individual outputs from just one pipleine like so. This fetches outputs from the neutrino pipleine, which are 12 maps.

In [ ]:
maps = model.distribution_makers[0].pipelines[0].get_outputs()

In [ ]:
maps.names

In [ ]:
fig, axes = plt.subplots(3,4, figsize=(24,10))
plt.subplots_adjust(hspace=0.5)
axes = axes.flatten()

for m, ax in zip(maps, axes):
    m.plot(ax=ax)

If we are interested in just the total expecatation from the full model (all neutrinos + muons), we can do the following:

In [ ]:
for o in model.get_outputs(return_sum=True):
    o.plot()

## Diff plots

Let's explore how a change in one of our nuisance parameters affects the expected counts per bin. Here we choose a *hole ice* parameter and move it a smidge.

In [ ]:
# reset all free parameters to put them back to nominal values
model.reset_free()
nominal = model.get_outputs(return_sum=True)

# shift one parameter
model.params.opt_eff_lateral.value = 20
model.update_params(model.params)
sys = model.get_outputs(return_sum=True)

In [ ]:
for i in range(len(model.distribution_makers)):
    ((nominal[i][0] - sys[i][0])/nominal[i][0]).plot(symm=True, clabel="rel. difference")

## Get Data

We can load the real observed data too. This is a Pipeline with no free parameters, as the data is of course fixed. Similar to before we just use the DeepCore data twice.
NB: When developping a new analysis you will **not** be allowed to look at the data as we do here before the box opening (c.f. *blindness*).

In [ ]:
# real data
data_maker1 = Pipeline("settings/pipeline/IceCube_3y_data.cfg")
data_maker1.detector_name = 'detector1'

data_maker2 = Pipeline("settings/pipeline/IceCube_3y_data.cfg")
data_maker2.detector_name = 'detector2'

In [ ]:
data_maker1

In [ ]:
data_maker2

In [ ]:
data = [data_maker1.get_outputs(), data_maker2.get_outputs()]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20, 7))
plt.subplots_adjust(hspace=0.5)

model.reset_free()
nominal = model.get_outputs(return_sum=True)

for i in range(len(data)):
    data[i].plot(ax=ax[i,0], title="Data")
    nominal[i].plot(ax=ax[i,1], title="Model")
    (data[i] - nominal[i]).plot(ax=ax[i,2], symm=True, title="Diff")

## Fitting

For fitting we need to configure a minimizer, several standard cfgs are available, but you can also define your own.
For the fit we need to choose a `metric`, and by default, theta23 octants, which are quasi degenerate, are fit seperately, which means two fits are run.

In [ ]:
ana = Analysis()

In [ ]:
# global minimizer
nlopt_settings = {
    "method": "nlopt",
    "method_kwargs": {
        "algorithm": "NLOPT_GN_CRS2_LM",
        "ftol_abs": 1e-3,
        "ftol_rel": 1e-3,
        # other options that can be set here: 
        # xtol_abs, xtol_rel, stopval, maxeval, maxtime
        # after maxtime seconds, stop and return best result so far
        "maxtime": 500
    },
    "local_fit_kwargs": None  # no further nesting available
}

# local minimizer
local_fit_minuit = {
    "method": "iminuit",
    "method_kwargs": {
        "errors": 0.1,
        "precision": 1e-14,  # default: double precision
        "tol": 1e-2,  # default: 0.1
        "run_simplex": False,
        "run_migrad": True
    },
    "local_fit_kwargs": None
}

# octant fit for local minimizer
fit_octant = {
    "method": "octants",
    "method_kwargs": {
        "angle": "theta23",
        "inflection_point": 45 * ureg.degrees,
    },
    "local_fit_kwargs": local_fit_minuit
}

# complete fit routine
fit_combi = {
    "method": "staged",
    "method_kwargs": None,
    "local_fit_kwargs":[
        nlopt_settings,
        fit_octant,
    ]
}

In [ ]:
%%time
result = ana.fit_recursively(
            data_dist=data,
            hypo_maker=model,
            metric=["mod_chi2", "mod_chi2"],
            external_priors_penalty=None,
            **fit_octant
        )

Here we can view the bestfit parameters - the result of our fit.

In [ ]:
bestfit_params = result['params'].free
bestfit_params

In [ ]:
# update the model with the bestfit (make a copy here, because we don't want our bestfit params to be affected (NB: stuff is passed by reference in python))
model.update_params(copy.deepcopy(bestfit_params))

Let's see how good that fit looks like. We here construct signed mod_chi2 maps by hand.
You can see that after the fit, it improved considerably, and the distribution of chi2 values is now more uniform - not much features can be seen anymore.

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(20, 14))
plt.subplots_adjust(hspace=0.5)

bestfit = model.get_outputs(return_sum=True)

for i in range(len(data)):
    data[i].plot(ax=ax[2*i,0], title="Data")
    nominal[i].plot(ax=ax[2*i,1], title="Nominal")
    diff = data[i] - nominal[i]
    (abs(diff)*diff/(nominal[i] + unp.std_devs(nominal[i].hist['total']))).plot(ax=ax[2*i,2], symm=True, title=r"signed $\chi^2$", vmin=-12, vmax=12)

    data[i].plot(ax=ax[2*i+1,0], title="Data")
    bestfit[i].plot(ax=ax[2*i+1,1], title="Bestfit")
    diff = data[i] - bestfit[i]
    (abs(diff)*diff/(bestfit[i] + unp.std_devs(bestfit[i].hist['total']))).plot(ax=ax[2*i+1,2], symm=True, title=r"signed $\chi^2$", vmin=-12, vmax=12)

When checking the chi2 value from the fitted model, you maybe see that it is around 113, while in the minimizer loop we saw it converged to 116. It is important to keep in mind that in the fit we had extended the metric with prior penalty terms. When we add those back we get the identical number as reported in the fit.

In [ ]:
print('detector1')
print(data[0].metric_total(nominal[0], 'mod_chi2'))
print(data[0].metric_total(bestfit[0], 'mod_chi2'))
print('-----')
print('detector2')
print(data[1].metric_total(nominal[1], 'mod_chi2'))
print(data[1].metric_total(bestfit[1], 'mod_chi2'))

Evaluating other metrics just for fun (and just for detector1):

In [ ]:
for metric in pisa.utils.stats.ALL_METRICS:
    try:
        print('%s = %.3f'%(metric, data[0].metric_total(bestfit[0], metric)))
    except:
        print('%s failed'%metric)

Adding two detectors and prior penalty terms

In [ ]:
data[0].metric_total(bestfit[0], 'mod_chi2') + data[1].metric_total(bestfit[1], 'mod_chi2') + model.params.priors_penalty('mod_chi2')

In [ ]:
result['metric_val']